In [1]:
import json
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [2]:
# boilerplate code to connect to mysql server
username = 'xqi'
password = 'xqi_1234!'
host = '52.6.148.1'       
port = 3306              
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}')

In [3]:
# load station dict from data/station_dict.json as a dictionary
with open('data/station_dict.json', 'r') as file:
    data = json.load(file)
data

{'1836026195': 'INDIAN RIVER INLET Lvl',
 '1836026194': 'INDIAN RIVER AT ROSEDALE BEACH Lvl',
 '271420501': 'LEWES Lvl',
 '1836026192': 'UNNAMED DITCH ON FRED HUDSON ROAD Lvl',
 '1836026191': 'LITTLE ASSAWOMAN BAY Lvl',
 '1836026193': 'VINES CREEK Lvl',
 '1413617665': 'REHOBOTH BAY AT DEWEY BEACH, DE Lvl'}

In [4]:
data.keys()  # should be a list of sensor ids

dict_keys(['1836026195', '1836026194', '271420501', '1836026192', '1836026191', '1836026193', '1413617665'])

In [5]:
def clean_hydro_data(df):
    """
    Clean hydro data by:
    1. Setting seconds to 0 for all collect_time values
    2. Removing duplicate timestamps
    3. Sorting by time
    4. Basic data validation
    
    Args:
        df: DataFrame with 'collect_time' column
    
    Returns:
        cleaned DataFrame
    """
    print(f"Original data shape: {df.shape}")
    print(f"Original time range: {df['collect_time'].min()} to {df['collect_time'].max()}")
    
    # Make a copy to avoid modifying original data
    df_cleaned = df.copy()
    
    # Convert collect_time to datetime if it's not already
    if df_cleaned['collect_time'].dtype == 'object':
        df_cleaned['collect_time'] = pd.to_datetime(df_cleaned['collect_time'])
    
    # Step 1: Set seconds to 0 (normalize to minute precision)
    print("\nStep 1: Normalizing collect_time to minute precision (setting seconds to 0)")
    original_times = df_cleaned['collect_time'].head(5)
    print(f"Original times (first 5): {list(original_times)}")
    
    df_cleaned['collect_time'] = df_cleaned['collect_time'].dt.floor('min')
    
    normalized_times = df_cleaned['collect_time'].head(5)
    print(f"Normalized times (first 5): {list(normalized_times)}")
    
    # Step 2: Check for and remove duplicates
    print(f"\nStep 2: Checking for duplicate timestamps")
    duplicates_before = df_cleaned.duplicated(subset=['collect_time']).sum()
    print(f"Found {duplicates_before} duplicate timestamps")
    
    if duplicates_before > 0:
        # Show some examples of duplicates
        duplicate_times = df_cleaned[df_cleaned.duplicated(subset=['collect_time'], keep=False)]['collect_time'].value_counts().head(3)
        print(f"Most common duplicate times (top 3):")
        for time, count in duplicate_times.items():
            print(f"  {time}: {count} occurrences")
            
        # Remove duplicates (keep first occurrence)
        df_cleaned = df_cleaned.drop_duplicates(subset=['collect_time'], keep='first')
        print(f"Removed {duplicates_before} duplicate timestamps")
    
    # Step 3: Sort by time
    print(f"\nStep 3: Sorting by collect_time")
    df_cleaned = df_cleaned.sort_values('collect_time').reset_index(drop=True)
    
    # Step 4: Basic validation and statistics
    print(f"\nStep 4: Data validation and statistics")
    print(f"Cleaned data shape: {df_cleaned.shape}")
    print(f"Cleaned time range: {df_cleaned['collect_time'].min()} to {df_cleaned['collect_time'].max()}")
    
    # Check for missing values
    missing_values = df_cleaned.isnull().sum()
    print(f"Missing values per column:")
    for col, missing in missing_values.items():
        if missing > 0:
            print(f"  {col}: {missing} ({missing/len(df_cleaned)*100:.2f}%)")
    
    # Time series continuity check
    time_diffs = df_cleaned['collect_time'].diff()
    most_common_interval = time_diffs.mode().iloc[0] if len(time_diffs.mode()) > 0 else None
    print(f"Most common time interval: {most_common_interval}")
    
    # Check for large gaps
    large_gaps = time_diffs[time_diffs > pd.Timedelta(hours=1)]
    if len(large_gaps) > 0:
        print(f"Found {len(large_gaps)} time gaps larger than 1 hour")
        print(f"Largest gap: {large_gaps.max()}")
    
    print(f"\nData cleaning completed!")
    print(f"Reduction: {len(df) - len(df_cleaned)} rows removed ({(len(df) - len(df_cleaned))/len(df)*100:.2f}%)")
    
    return df_cleaned

print("Data cleaning function defined!")

Data cleaning function defined!


In [6]:
for key in data.keys():
    print(f"Sensor ID: {key}, Station Name: {data[key]}")
    sensor_id = key
    query = f"""
    select * from deep_learning.hydro_data
    where sensor_id = {sensor_id}
    and collect_time > "2024-07-01" 
    and collect_time < "2025-6-30"
    and uom ="ft"
    order by collect_time;
    """
    df = pd.read_sql(query, engine)
    df = clean_hydro_data(df)
    df['seconds'] = df['collect_time'].astype('int64') // 10**9
    df.to_parquet(f'data/{sensor_id}.parquet', index=False)

Sensor ID: 1836026195, Station Name: INDIAN RIVER INLET Lvl
Original data shape: (98642, 6)
Original time range: 2024-07-01 00:02:15 to 2025-06-29 23:57:18

Step 1: Normalizing collect_time to minute precision (setting seconds to 0)
Original times (first 5): [Timestamp('2024-07-01 00:02:15'), Timestamp('2024-07-01 00:02:15'), Timestamp('2024-07-01 00:08:15'), Timestamp('2024-07-01 00:14:15'), Timestamp('2024-07-01 00:20:15')]
Normalized times (first 5): [Timestamp('2024-07-01 00:02:00'), Timestamp('2024-07-01 00:02:00'), Timestamp('2024-07-01 00:08:00'), Timestamp('2024-07-01 00:14:00'), Timestamp('2024-07-01 00:20:00')]

Step 2: Checking for duplicate timestamps
Found 16612 duplicate timestamps
Most common duplicate times (top 3):
  2025-05-01 16:56:00: 7 occurrences
  2024-12-16 16:17:00: 4 occurrences
  2024-12-16 14:47:00: 4 occurrences
Removed 16612 duplicate timestamps

Step 3: Sorting by collect_time

Step 4: Data validation and statistics
Cleaned data shape: (82030, 6)
Cleaned 

In [7]:
df.head(10)

,id,collect_time,date_created,sensor_id,uom,value,seconds
0,9247853,2025-03-26 13:36:00,2025-03-26 14:15:45.261,1413617665,ft,0.14,1742996160
1,9248869,2025-03-26 14:36:00,2025-03-26 14:55:45.297,1413617665,ft,-0.11,1742999760
2,9250266,2025-03-26 15:36:00,2025-03-26 15:50:45.240,1413617665,ft,-0.31,1743003360
3,9251155,2025-03-26 16:12:00,2025-03-26 16:25:45.286,1413617665,ft,-0.01,1743005520
4,9251790,2025-03-26 16:24:00,2025-03-26 16:50:45.294,1413617665,ft,-0.29,1743006240
5,9252552,2025-03-26 17:06:00,2025-03-26 17:20:45.258,1413617665,ft,0.00,1743008760
6,9252679,2025-03-26 17:12:00,2025-03-26 17:25:45.303,1413617665,ft,0.00,1743009120
7,9253314,2025-03-26 17:30:00,2025-03-26 17:50:46.633,1413617665,ft,-0.02,1743010200
8,9253441,2025-03-26 17:42:00,2025-03-26 17:55:45.680,1413617665,ft,0.02,1743010920
9,9254711,2025-03-26 18:30:00,2025-03-26 18:45:45.231,1413617665,ft,0.05,1743013800


In [13]:
import pandas as pd

df = pd.read_parquet('data/1836026194.parquet')
df = df[(df['collect_time'] > '2024-07-02') & (df['collect_time'] < '2024-07-03')]
df


,id,collect_time,date_created,sensor_id,uom,value,seconds
240,204767,2024-07-02 00:06:00,2024-07-02 00:10:46.417,1836026194,ft,-0.04,1719878760
241,204896,2024-07-02 00:12:00,2024-07-02 00:15:46.495,1836026194,ft,-0.08,1719879120
242,205151,2024-07-02 00:18:00,2024-07-02 00:25:46.419,1836026194,ft,-0.11,1719879480
243,205280,2024-07-02 00:24:00,2024-07-02 00:30:46.734,1836026194,ft,-0.15,1719879840
244,205409,2024-07-02 00:30:00,2024-07-02 00:35:46.671,1836026194,ft,-0.18,1719880200
...,...,...,...,...,...,...,...
472,240720,2024-07-02 23:30:00,2024-07-02 23:35:46.669,1836026194,ft,0.81,1719963000
473,240849,2024-07-02 23:36:00,2024-07-02 23:40:46.482,1836026194,ft,0.76,1719963360
474,240978,2024-07-02 23:42:00,2024-07-02 23:45:46.440,1836026194,ft,0.71,1719963720
475,241236,2024-07-02 23:48:00,2024-07-02 23:55:46.456,1836026194,ft,0.65,1719964080


In [15]:
df.to_csv('data/hydro_data_1836026195.csv', index=False)